# SVM기반 분류모델_당뇨병여부(지체장애인 유의미한 변수) 2차 시도

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVC
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV

In [61]:
df=pd.read_excel("pydata/[로지스틱] 유의한 변수 추출(지체장애 당뇨병)ver2.xlsx")
df.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,16)만성질환명(골관절염(퇴행성_x000D_\n관절염)),17)만성질환명(류마티스 관절염),18)만성질환명(골다공증),19)만성질환명(척추측만증),20)만성질환명(허리목통증),21)만성질환명(피부염),22)만성질환명(백내장),23)만성질환명(우울증),24)만성질환명(암),25)만성질환명(기타)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,0,0,0,0,0,0,0,0,0,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,0,0,0,0,0,0,0,0,0,1
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,0,0,0,0,1,0,0,0,0,1
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,0,0,0,0,1,0,0,1,0,0
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,1,0,0,0,0,0,0,0,0,0


In [23]:
df.columns

Index(['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부', '01)만성질환명(고혈압)', '02)만성질환명(뇌졸중,중풍)',
       '03)만성질환명(심근경색증)', '04)만성질환명(협심증)', '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)',
       '07)만성질환명(갑상선장애)', '08)만성질환명(천식)', '09)만성질환명(폐결핵)',
       '10)만성질환명(폐질환(만성기관 지염,폐기종)', '11)만성질환명(위십이지장궤양)', '12)만성질환명(B형간염)',
       '13)만성질환명(C형간염)', '14)만성질환명(간경변증)', '15)만성질환명(신부전)',
       '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))', '17)만성질환명(류마티스 관절염)',
       '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)', '20)만성질환명(허리목통증)', '21)만성질환명(피부염)',
       '22)만성질환명(백내장)', '23)만성질환명(우울증)', '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [3]:
df1=df[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부',
       '06)만성질환명(당뇨병)']]
df1 # 2724건 

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,1935,0,157.0,65.0,11,68,0,2,0,2,...,1,2,2,2,2,0,2,2,2,0
2720,1952,0,165.0,62.0,11,69,0,1,4,5,...,1,2,2,2,2,0,0,0,1,0
2721,1960,30,173.0,88.0,11,48,175,2,5,4,...,1,2,2,1,2,0,5,3,2,0
2722,1960,0,153.0,51.0,11,48,175,2,0,2,...,1,2,2,1,2,0,3,4,2,0


### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [4]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [5]:
out=df1.loc[[100]] # 유: 4, 27 / 무: 46, 100
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
100,1967,0,164.0,72.0,21,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,0


In [6]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [7]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [8]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [9]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2020
1                 703
dtype: int64

In [138]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=123, stratify=y_data)

In [139]:
y_train.value_counts(), y_test.value_counts()

(06)만성질환명(당뇨병)
 0                1415
 1                 491
 dtype: int64,
 06)만성질환명(당뇨병)
 0                606
 1                211
 dtype: int64)

In [140]:
df.columns

Index(['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부', '01)만성질환명(고혈압)', '02)만성질환명(뇌졸중,중풍)',
       '03)만성질환명(심근경색증)', '04)만성질환명(협심증)', '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)',
       '07)만성질환명(갑상선장애)', '08)만성질환명(천식)', '09)만성질환명(폐결핵)',
       '10)만성질환명(폐질환(만성기관 지염,폐기종)', '11)만성질환명(위십이지장궤양)', '12)만성질환명(B형간염)',
       '13)만성질환명(C형간염)', '14)만성질환명(간경변증)', '15)만성질환명(신부전)',
       '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))', '17)만성질환명(류마티스 관절염)',
       '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)', '20)만성질환명(허리목통증)', '21)만성질환명(피부염)',
       '22)만성질환명(백내장)', '23)만성질환명(우울증)', '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [141]:
# 범주형 변수: 원핫 인코딩 수행(MaxAbsScaler: 최대절대값과 0이 각각 1, 0이 되도록 스케일링. 즉, 즉 -1~1 사이로 재조정)
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_train)
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [142]:
#모델 생성
model = SVC(kernel='rbf', C=10.0, gamma=0.1, probability=True)#,class_weight=class_weight)

In [143]:
#모델 학습
model.fit(x_train, y_train)

#모델 검증
print('training data fit :', model.score(x_train, y_train)) 
print('test data fit :', model.score(x_test, y_test)) 

# with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
#     pickle.dump(model, f)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training data fit : 0.8058761804826863
test data fit : 0.7086903304773562


In [144]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1) # out 데이터에서 '고혈압' 칼럼만 제거
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,1,2,2,2,2,0,0,0,2


In [145]:
labels = ['무', '유']

In [146]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.732086206840701


---
---

### 오버샘플링_SVC

In [286]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [640]:
out=df1.loc[[4]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [641]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1938,30,155.0,60.0,11,63,0,1,0,2,...,1,2,2,2,2,0,1,4,2,0


In [642]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [643]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [644]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [645]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [646]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [647]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [343]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [344]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [244]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [648]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [649]:
#모델 생성
model = SVC(kernel='rbf', C=1.0, gamma=0.1, probability=True) #,class_weight=class_weight)

#모델 학습
model.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수: {:.4f}'.format(model.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format( model.score(x_test, y_test)))

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


학습 데이터 점수: 0.9223
평가 데이터 점수 :0.7013


In [650]:
# 그리드 서치
## 비대칭 매개변수 그리드 탐색
param_grid = [ {'kernel': ['rbf', 'ploy', 'sigmoid'], # 가장 성능이 좋은 것: 가우시안 RBF 커널
             'C': [0.01, 0.1, 1, 10], # 값이 클수록 이상치의 존재 가능성을 작게 봐서 좀 더 세심하게 결정 경계를 찾아낸다. -> 오버피팅(하드마진)
             'gamma': [0.01, 0.1, 1, 10]},  # 값이 클수록 결정 경계의 곡률이 커짐 -> 오버피팅
             {'kernel': ['linear'],
              'C': [0.01, 0.1, 1, 10]
             } ]

gs = GridSearchCV(estimator=SVC(probability=True), 
                    param_grid=param_grid, scoring='accuracy', cv = 5, 
                    n_jobs=-1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_))
print('최적 점수: {:.4f}'.format(gs.best_score_))
print(result.best_estimator_)

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 260.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 342, in _sparse_fit
    kernel_type = self._sparse_kernels.index(kernel)
Va

최적 파라미터: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
최적 점수: 0.7951
SVC(C=1, gamma=0.1, probability=True)


In [651]:
#모델 학습
svc_model = gs.best_estimator_

In [652]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1)
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,1,4,2,1,2,2,0,2,9,2


In [653]:
labels = ['무', '유']

In [654]:
x_test = transformer.transform(x_test)

y_predict = svc_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = svc_model.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.8869


---
### 오버샘플링_LightGBM

In [682]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [683]:
out=df1.loc[[4]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [684]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1938,30,155.0,60.0,11,63,0,1,0,2,...,1,2,2,2,2,0,1,4,2,0


In [685]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [686]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [687]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [688]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [689]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [690]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [691]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [692]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [693]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [694]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [695]:
#모델 생성
import lightgbm as lgbm
lgb_model= lgbm.LGBMClassifier()

#모델 학습
lgb_model.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수: {:.4f}'.format(lgb_model.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format(lgb_model.score(x_test, y_test)))

학습 데이터 점수: 0.9159
평가 데이터 점수 :0.6732


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [696]:
# 그리드 서치
param_grid = {'learning_rate': [0.05, 0.1, 0.2],
             'max_depth': [3, 5, 7]}
gs = GridSearchCV(estimator=lgbm.LGBMClassifier(), param_grid=param_grid, cv=5, n_jobs=-1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'learning_rate': 0.2, 'max_depth': 5}
최적 점수: 0.7399
LGBMClassifier(learning_rate=0.2, max_depth=5)


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [697]:
#모델 학습
lgb_model = gs.best_estimator_

In [698]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1)
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,1,4,2,1,2,2,0,2,9,2


In [699]:
labels = ['무', '유']

In [700]:
x_test = transformer.transform(x_test)

y_predict = lgb_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = lgb_model.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)
print(label, confidence) 

무 0.815


---
### 오버샘플링_Decision Tree

In [447]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [655]:
out=df1.loc[[27]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1


In [656]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [657]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [658]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [659]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [660]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [661]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [662]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [663]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [489]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [490]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [664]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [665]:
from sklearn.tree import DecisionTreeClassifier

#모델 학습
df_clf = DecisionTreeClassifier()
df_clf.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(df_clf.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format(df_clf.score(x_test, y_test)))

학습 데이터 점수 :1.0000
평가 데이터 점수 :0.5924


In [666]:
# 그리드 서치
param_grid = {'splitter': ['random', 'best'],
              'max_depth': [3, 5, 7, 9],
              'min_samples_split': [1, 3, 5], # 노드를 분할하기 위한 최소한의 샘플 수
              'min_samples_leaf': [1, 3, 5]} # 리프 노드가 될 수 있는 샘플 데이터의 최소 개수

gs = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=param_grid, cv = 5, n_jobs=-1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'random'}
최적 점수: 0.6473
DecisionTreeClassifier(max_depth=9, min_samples_split=3, splitter='random')


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
120 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0

In [667]:
#모델 학습
df_clf = gs.best_estimator_

In [668]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1)
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,1,2,2,2,2,0,0,0,2


In [669]:
labels = ['무', '유']

In [670]:
x_test = transformer.transform(x_test)

y_predict = df_clf.predict(x_test)
label = labels[y_predict[0]]
y_predict = df_clf.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.7312


---
### 오버샘플링_KNN

In [498]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [622]:
out=df1.loc[[26]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1


In [623]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [624]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [625]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [626]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [627]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [628]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [629]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [630]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [631]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [632]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [633]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [634]:
from sklearn.neighbors import KNeighborsClassifier

#모델 학습
knn_model = KNeighborsClassifier()
knn_model.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(knn_model.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format(knn_model.score(x_test, y_test)))

C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


학습 데이터 점수 :0.7240
평가 데이터 점수 :0.4272


In [635]:
# 그리드 서치
param_grid = {'n_neighbors': range(1, 16),
             'weights': ['uniform', 'distance'],
             'algorithm': ['ball_tree', 'kd_tree', 'brute'],
             'p': [1,2] # 1: 멘하튼 거리, 2: 유클리디안 거리
             }

gs = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, n_jobs=-1, cv = 5)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'algorithm': 'ball_tree', 'n_neighbors': 2, 'p': 1, 'weights': 'distance'}
최적 점수: 0.7092
KNeighborsClassifier(algorithm='ball_tree', n_neighbors=2, p=1,
                     weights='distance')


C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


In [636]:
#모델 학습
knn_model = gs.best_estimator_

In [637]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1)
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,1,2,2,2,0,5,7,2


In [638]:
labels = ['무', '유']

In [639]:
x_test = transformer.transform(x_test)

y_predict = knn_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = knn_model.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.5


---
### 오버샘플링_Logistic Regresstion

In [752]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [753]:
out=df1.loc[[4]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [754]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1938,30,155.0,60.0,11,63,0,1,0,2,...,1,2,2,2,2,0,1,4,2,0


In [755]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [756]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [757]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [758]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [759]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [760]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [761]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [744]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [721]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [762]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [763]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(log_model.score(x_sm, y_sm)))
print('평가 데이터 점수:{:.4f}'.format(log_model.score(x_test, y_test)))

학습 데이터 점수 :0.8339
평가 데이터 점수:0.6916


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [764]:
# 그리드 서치
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), categorical_features), remainder='passthrough')
# # transformer = make_pipeline(transformer, MaxAbsScaler())

# pipe = Pipeline([('scaler', transformer),
#                  ('smote', SMOTE()),
#                 ('model', SVC())])

param_grid = [{'penalty' : ['l1', 'l2'],
               'C' : [0.001, 0.01, 0.1, 1, 10, 100]}]

gs = GridSearchCV(estimator = LogisticRegression(), param_grid = param_grid,
                  scoring = 'accuracy', cv = 5, n_jobs = -1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'C': 10, 'penalty': 'l2'}
최적 점수: 0.7664
LogisticRegression(C=10)


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueErr

In [765]:
#모델 학습
log_model = gs.best_estimator_

In [766]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1) # out 데이터에서 '당뇨병' 칼럼만 제거
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,1,4,2,1,2,2,0,2,9,2


In [767]:
labels = ['무', '유']

In [768]:
x_test = transformer.transform(x_test)

y_predict = log_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = log_model.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.8695


---
### 오버샘플링_Random Forest

In [838]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [839]:
out=df1.loc[[4]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [840]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1938,30,155.0,60.0,11,63,0,1,0,2,...,1,2,2,2,2,0,1,4,2,0


In [841]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [842]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [843]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [844]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [845]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [846]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [847]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [848]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [490]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [849]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [850]:
from sklearn.ensemble import RandomForestClassifier

#모델 학습
df_clf = RandomForestClassifier()
df_clf.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(df_clf.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format(df_clf.score(x_test, y_test)))

<ipython-input-850-0e9d92697a09>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  df_clf.fit(x_sm, y_sm)


학습 데이터 점수 :1.0000
평가 데이터 점수 :0.6622


In [851]:
# 그리드 서치
param_grid = {'n_estimators': [100, 200, 300, 400, 500],
              'max_depth': [3, 5, 7, 9],
              'min_samples_split': [3, 5, 7], # 노드를 분할하기 위한 최소한의 샘플 수
              'min_samples_leaf': [3, 5, 7]} # 리프 노드가 될 수 있는 샘플 데이터의 최소 개수

gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid=param_grid, cv = 5, n_jobs=-1)

result = gs.fit(x_sm, y_sm)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최적 파라미터: {'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 400}
최적 점수: 0.6996
RandomForestClassifier(max_depth=9, min_samples_leaf=3, min_samples_split=3,
                       n_estimators=400)


In [852]:
#모델 학습
df_clf = gs.best_estimator_

In [853]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1)
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,1,4,2,1,2,2,0,2,9,2


In [854]:
labels = ['무', '유']

In [855]:
x_test = transformer.transform(x_test)

y_predict = df_clf.predict(x_test)
label = labels[y_predict[0]]
y_predict = df_clf.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.5547


---
### 오버샘플링_Soft Voting

In [912]:
df1[df1['06)만성질환명(당뇨병)']==1]

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1
7,1949,30,150.0,50.0,31,48,180,2,0,3,...,3,2,2,2,2,0,1,3,2,1
15,1967,0,161.0,70.0,24,40,103,2,0,1,...,2,2,1,2,2,0,3,4,2,1
26,1955,0,165.0,60.0,11,69,0,2,1,2,...,1,1,2,2,2,0,5,7,2,1
27,1944,0,168.0,74.0,23,69,0,2,1,4,...,1,2,2,2,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,1946,30,163.0,57.0,11,62,0,2,0,5,...,1,2,2,2,2,0,0,0,1,1
2713,1938,30,150.7,67.0,40,48,171,2,0,2,...,1,2,2,2,2,0,1,4,2,1
2714,1932,30,152.0,59.0,40,69,0,2,0,2,...,1,2,2,2,2,0,1,3,2,1
2715,1938,0,145.0,60.0,24,48,180,2,1,2,...,1,2,2,2,2,0,0,0,2,1


In [913]:
out=df1.loc[[4]] # 유: 4, 15, 27 / 무: 46, 100, 1111
out

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,4,2,1,2,2,0,2,9,2,1


In [914]:
df2=df1.drop(out.index).reset_index(drop=True) # 4번 인덱스 제거
df2.head()

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부,06)만성질환명(당뇨병)
0,1970,0,178.0,77.0,21,64,0,2,3,5,...,1,2,2,2,2,0,0,0,1,0
1,1949,30,163.0,55.0,21,69,0,1,3,4,...,1,2,2,2,2,0,0,0,2,0
2,1958,0,158.0,63.0,11,48,175,2,0,4,...,1,2,2,2,2,0,0,0,2,0
3,1955,30,163.0,62.0,29,48,175,2,2,4,...,1,2,2,2,2,0,1,3,2,0
4,1938,30,155.0,60.0,11,63,0,1,0,2,...,1,2,2,2,2,0,1,4,2,0


In [915]:
# 독립변수 테이블 생성
x_data=df2[['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
       '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
       'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
       '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']]

In [916]:
# 종족변수 테이블 생성
y_data=df2[['06)만성질환명(당뇨병)']]

In [917]:
y_data.value_counts() # 유무 빈도 확인

06)만성질환명(당뇨병)
0                2021
1                 702
dtype: int64

In [918]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=12, stratify=y_data)

In [919]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [920]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [921]:
y_sm.value_counts() # 비율 1:1 확인

06)만성질환명(당뇨병)
0                1415
1                1415
dtype: int64

In [900]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [879]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['생년', '월 혈압약 일수', '키', '몸무게', '가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부',
#        '평균음주량', '음식물 씹는데 불편감 여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리',
#        'ADL_보행', '상지의지-소지', '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용',
#        '사용빈도', '1일 이용시간', '컴퓨터 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [922]:
numeric_features = ['생년', '월 혈압약 일수', '키', '몸무게']
categorical_features = ['가구 유형', '장애주된 원인', '질병명', '물리치료-이용여부', '평균음주량', '음식물 씹는데 불편감 여부',
        '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', 'ADL_보행', '상지의지-소지',
        '맞춤형 교정용신발-필요', '자세보조용구-필요', '이동변기-소지', '이동변기-사용', '사용빈도', '1일 이용시간',
        '컴퓨터 사용 여부']

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

preprocessor.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

# # 모델 생성
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('svc', SVC(probability=True))])

# #모델 학습
# clf.fit(x_sm, y_sm)

# #모델 검증
# print('training data fit :', clf.score(x_sm, y_sm)) 

# print('test data fit :', clf.score(x_test, y_test))

In [902]:
from sklearn.ensemble import VotingClassifier

In [903]:
RFC = RandomForestClassifier()
 
## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [3, 8, 8],
              "min_samples_split": [2, 3, 8],
              "min_samples_leaf": [1, 3, 8],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}
 
gsRFC = GridSearchCV(RFC, rf_param_grid, cv=5, scoring="accuracy", n_jobs = -1)
#print(score)
 
gsRFC.fit(x_sm, y_sm)
 
RFC_best = gsRFC.best_estimator_
 
# Best score
gsRFC.best_score_

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


0.7809187279151943

In [904]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }
 
gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=5, scoring="accuracy", n_jobs = -1)
 
gsGBC.fit(x_sm, y_sm)
 
GBC_best = gsGBC.best_estimator_
 
# Best score
gsGBC.best_score_

C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6901060070671379

In [923]:
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}
 
gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=5, scoring="accuracy", n_jobs= -1)
 
gsSVMC.fit(x_sm, y_sm)
 
SVMC_best = gsSVMC.best_estimator_
 
# Best score
gsSVMC.best_score_

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7858657243816255

In [906]:
from xgboost import XGBClassifier
XGBC = XGBClassifier()
xgb_param_grid = {'max_depth':[3,5,7],
                  'min_child_weight':[3,5,6],
                  'gamma': [ 0, 0.001, 0.01, 0.1, 1],
                  'learning_rate':[0.1, 0.05, 0.01]}
 
gsXGBC = GridSearchCV(XGBC,param_grid = xgb_param_grid, cv=5, scoring="accuracy", n_jobs= -1)
 
gsXGBC.fit(x_sm, y_sm)
 
XGBC_best = gsXGBC.best_estimator_
 
# Best score
gsXGBC.best_score_

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[14:52:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7332155477031802

In [907]:
#Voting Classifier - voting 파라미터로 hard/soft 선택가능
votingC = VotingClassifier(
    estimators=[('rfc', RFC_best), ('svc', SVMC_best),('gbc',GBC_best), ('xgb', XGBC_best)], 
    voting='soft', # 간접 투표
    n_jobs=4
)

# 모델 학습
votingC = votingC.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(votingC.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format(votingC.score(x_test, y_test)))

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


학습 데이터 점수 :0.9735
평가 데이터 점수 :0.6928


In [908]:
x_test=out.drop(['06)만성질환명(당뇨병)'],axis=1)
x_test

,생년,월 혈압약 일수,키,몸무게,가구 유형,장애주된 원인,질병명,물리치료-이용여부,평균음주량,음식물 씹는데 불편감 여부,...,이용_만성질환관리,ADL_보행,상지의지-소지,맞춤형 교정용신발-필요,자세보조용구-필요,이동변기-소지,이동변기-사용,사용빈도,1일 이용시간,컴퓨터 사용 여부
4,1936,0,154.0,57.0,24,48,171,2,0,2,...,1,4,2,1,2,2,0,2,9,2


In [909]:
labels = ['무', '유']

In [910]:
x_test = transformer.transform(x_test)

y_predict = votingC.predict(x_test)
label = labels[y_predict[0]]
y_predict = votingC.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.6914
